In [190]:
import os 

os.environ["TMPDIR"] = "/tmp"

from qat.interop.qiskit import QPUToBackend,AsyncBackendToQPU,qiskit_to_qlm
from qat.qpus import PyLinalg,get_default_qpu
from qat.plugins import ScipyMinimizePlugin as ScipyMinimizePluginLocal
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal

#%%pycodestyle
from qiskit_nature.drivers import UnitsType
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.algorithms import VQEUCCFactory,GroundStateEigensolver

from qiskit_nature.circuit.library import UCCSD,UCC
from qiskit_nature.circuit.library.initial_states import HartreeFock
from qiskit.algorithms.optimizers import SLSQP,L_BFGS_B,SPSA,COBYLA
from qiskit.utils import QuantumInstance
from qiskit import Aer

import time as time

import qiskit
print(qiskit.__qiskit_version__)
print(qiskit.__version__)




{'qiskit-terra': '0.18.2', 'qiskit-aer': '0.8.2', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.16.0', 'qiskit-aqua': '0.9.5', 'qiskit': None, 'qiskit-nature': '0.2.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}
0.18.2


In [191]:
remote =True
if remote:
    from qlmaas.qpus import LinAlg
    from qlmaas.plugins import Nnizer,ScipyMinimizePlugin
    from qlmaas.plugins import IterativeExplorationVQE as IterativeExplorationVQEremote


    # Set up 3 different backends to test computation time
    backend_remoteQLM = ['remote backend QLM', QPUToBackend(qpu=LinAlg(), optimizer=ScipyMinimizePlugin(method="COBYLA"))]
backend_localQLM = ['local backend QLM', QPUToBackend(qpu=PyLinalg() , optimizer=ScipyMinimizePluginLocal(method="COBYLA"))]
backend_localQiskit = ['local backend qiskit', Aer.get_backend('aer_simulator')]
shots=1024

### OLD Method for integrating qiskit-qlm
Simply swap the backend you pass to Quantuminstance with a qpu from qlm (i.e. backendQLM=QPUToBackend(qpu))

In [192]:
# Initialize qubit converter


qubit_converter = QubitConverter(mapper=JordanWignerMapper(), two_qubit_reduction=True)  # two qubit reduction doesn`t work with Jordan-WignerMapper()
optimizer = COBYLA(maxiter=100)
bond_distance = 0.735
# Molecule
molecule = 'H .0 .0 -{0}; H .0 .0 {0}'
molecule_set = molecule.format(bond_distance/2)
driver = PySCFDriver(atom=molecule.format(bond_distance/2), unit=UnitsType.ANGSTROM, basis='sto3g')
qmolecule = driver.run()
es_problem = ElectronicStructureProblem(driver,q_molecule_transformers=[FreezeCoreTransformer(freeze_core=True, remove_orbitals=[])])



None


In [193]:

num_particles = qmolecule.num_alpha,qmolecule.num_beta
hf_init = HartreeFock(qubit_converter = qubit_converter, 
                      num_spin_orbitals = qmolecule.num_molecular_orbitals*2, 
                      num_particles = num_particles )
#ANSATZES TO TRY
uc_circuit = UCCSD(qubit_converter = qubit_converter,
                   initial_state=hf_init) #doesnt work with conversion to QLM #now it does
tl_circuit = TwoLocal(4, ['ry', 'rz'], 'cx',
                      entanglement='full', reps=1)

ansatz_in_use = uc_circuit #<----select type of ansatz

print('Results from simpler method of swapping backends')
for label,backend in [backend_localQiskit]:#,backend_localQLM, backend_remoteQLM]:
    quantum_instance = QuantumInstance(backend=backend, shots=shots,  max_credits=None)
    vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
    calc = GroundStateEigensolver(qubit_converter, vqe_solver)
    start = time.time()
    res = calc.solve(es_problem)
    end = time.time()
    #print('Type of backend:',label,', VQE result:\t', res.total_energies,',exec_time=', end - start)


# Compute exact solution
from qiskit.algorithms import NumPyMinimumEigensolver

# numpy_solver = NumPyMinimumEigensolver()
# exact_calc = GroundStateEigensolver(qubit_converter, numpy_solver)
# res_exact = exact_calc.solve(es_problem)
# print('Exact:\t', res_exact.total_energies)

Results from simpler method of swapping backends
None


### Newer method for integrating qiskit-qlm
The idea is to first modify the function that extract the energy from the quantum machine to send jobs to qlm,
Then we embed the GroundStateSolver in a custom QLM Plugin() (acutally is a Junction()), and together with the QLM qpu we build a stack.
This stack is then submitted with a fake job (this is required for qlm)

In [194]:

plugin = IterativeExplorationVQEremote(converter=qubit_converter, solver=vqe_solver, molecule=molecule_set )

In [195]:
# NEW METHOD
from qat.interop.qiskit import qiskit_to_qlm
from wrapper2myqlm import simple_qlm_job
print('Results from more complex method of embedding the method in the qlm stack in conjuction with the qpu')
for label,backend in [backend_remoteQLM]:
    quantum_instance = QuantumInstance(backend=backend, shots=shots,  max_credits=None)
    vqe_solver2 = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
    calc2 = GroundStateEigensolver(qubit_converter, vqe_solver2)
    #calc2.solve(es_problem) #this cause multiple jobs in qlm, wtf
    # stack = build_QLM_stack_remote(calc, molecule_set, IterativeExplorationVQEremote, qpu)
    #plugin = IterativeExplorationVQEremote(converter=qubit_converter, solver=vqe_solver, molecule=molecule_set )
    stack = plugin |  LinAlg()
    #stack = test | qpu
    start = time.time()
    async_result = stack.submit(simple_qlm_job(), meta_data={"optimal_parameters":'','qat':''})
    result = async_result.join()
    # solution = run_QLM_stack(stack)
    end = time.time()
    print('Type of backend:',label,' , VQE result:\t', result,',exec_time=', end - start)


Results from more complex method of embedding the method in the qlm stack in conjuction with the qpu
Submitted a new batch: Job3082
Type of backend: remote backend QLM  , VQE result:	 Result(raw_data=None, _value=ComplexNumber(re=-1.1373060208664287, im=0.0), error=None, value_data=None, error_data=None, meta_data={'optimal_parameters': "/home_nfs/gsilvi/.local/lib/python3.9/site-packages\n__['/usr/bin', '/usr/lib/python3.9/site-packages', '/usr/lib64/python39.zip', '/usr/lib64/python3.9/lib-dynload', '/home_nfs/gsilvi/custom_qlm_code', '/usr/local/lib/python3.9/site-packages/IPython/extensions', '/home_nfs/gsilvi/.local/lib/python3.9/site-packages', '/home_nfs/gsilvi/.local/lib/python3.9/site-packages/qat', '/usr/lib64/python3.9']_QAT:/home_nfs/gsilvi/.local/lib/python3.9/site-packages/qat/__init__.so", 'single_job': 'True'}, in_memory=None, data=None, qregs=[DefaultRegister(length=1, start=0, msb=None, key=None)], _parameter_map=None, need_flip=False, nbqbits=None, lsb_first=False, h

In [22]:
from importlib import import_module

interop = import_module('qat.interop',package='qat')
interop.qiskit.qlm_to_qiskit

<function qat.interop.qiskit.converters.qlm_to_qiskit(qlm_circuit, qubits=None)>

In [2]:
# import qat

import sys
import importlib.util; 
# spec = importlib.util.spec_from_file_location("qat","/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/site-packages/qat/__init__.py",submodule_search_locations=[''])
# module = importlib.util.module_from_spec(spec)

# spec.loader.exec_module(module)
# spec2 = importlib.util.spec_from_file_location("qat.interop","/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/site-packages/qat/interop/__init__.py",submodule_search_locations=[''])
# module2 = importlib.util.module_from_spec(spec2)

# spec2.loader.exec_module(module2)
spec3 = importlib.util.spec_from_file_location("qat","/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/site-packages/qat/__init__.py",submodule_search_locations=[])
module3 = importlib.util.module_from_spec(spec3)
sys.modules['qat'] = module3
spec3.loader.exec_module(module3)

sys.modules['qat2'] = module3


from qat2.interop.qiskit import qiskit_to_qlm

In [117]:
str(sys.path)

"['/home/gsilvi/Projects/NEASQ-4/Variationals_algorithms/misc/notebooks', '/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python39.zip', '/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9', '/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/lib-dynload', '', '/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/site-packages', '/home/gsilvi/Projects/NEASQ-4/Variationals_algorithms', '/home/gsilvi/miniforge3/envs/NEASQC4featMYQLM/lib/python3.9/site-packages/IPython/extensions', '/home/gsilvi/.ipython', '/home/gsilvi/.local/lib/python3.9/site-packages']"

In [41]:
# convert only the ansatz and pass the hamiltonian via matrix to Observables)
from qat.interop.qiskit import qiskit_to_qlm
from qat.core import Observable


from qiskit.compiler import transpile

def VQEwrapper_to_stack(groundstate_solver, es_problem, stack):
    #if isinstance(groundstate_solver._solver, MinimumEigensolverFactory):
    # this must be called after transformation.transform
    solver = groundstate_solver._solver.get_solver(es_problem, groundstate_solver._qubit_converter)
    #solver = groundstate_solver.solver
    ansatz_in_use = solver.ansatz
    #backend = solver.quantum_instance.backend
    backend = QPUToBackend(stack.qpu)

    transpiled_circ = transpile(ansatz_in_use.decompose(),
                                basis_gates = backend.configuration().basis_gates,
                                optimization_level = 0)
    qcirc = qiskit_to_qlm(transpiled_circ)    

    # Convert the observable/hamiltonian
    
    fermionic_hamiltonian = es_problem.second_q_ops()[0]
    hamiltonian_spin = groundstate_solver._qubit_converter.convert_match(fermionic_hamiltonian)
    num_qubits = hamiltonian_spin.num_qubits
    # print(num_qubits, hamiltonian_spin)
    job = qcirc.to_job(observable = Observable(num_qubits, matrix=hamiltonian_spin.to_matrix()))  

    # Stack to run locally (seems faster)
    
    start = time.time()
    calculation = stack.submit(job)
    end = time.time()

    #print("Final energy :", calculation.value , ' , exec time =', end - start)      
    # Convert to qiskit results format        
    raw_result = solver.compute_minimum_eigenvalue.__annotations__['return']()   
    raw_result.eigenvalue =  calculation.value
    raw_result.optimal_parameters = solver.ansatz.parameters
    dict_param = {}
    for i,par in enumerate(solver.ansatz.parameters):
        dict_param[par] = calculation.parameter_map[str(par)]
    raw_result.eigenstate = solver._get_eigenstate(dict_param)
    
    result = es_problem.interpret(raw_result)
    return result



#the problem
quantum_instance = QuantumInstance(backend=backend_localQiskit[1], shots=shots,  max_credits=None)
vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)

#the stack
stack = ScipyMinimizePluginLocal(method="COBYLA") | get_default_qpu()
start = time.time()
res2 = VQEwrapper_to_stack(groundstate_solver=calc , es_problem=es_problem , stack=stack)
end = time.time()

print('Result',res2.total_energies,' ,exec_time=', end - start)

None
Result [-1.13730603+0.j]  ,exec_time= 2.708992980999028


In [ ]:
from qiskit.algorithms import VQE
from qiskit.opflow import (
    OperatorBase,
    StateFn
)
import numpy as np
from typing import Optional, List, Callable, Union, Dict, Tuple
from qat.qlmaas.result import AsyncResult

class VQE_MyQLM(VQE):
    def get_energy_evaluation(
        self,
        operator: OperatorBase,
        return_expectation: bool = False,
    ) -> Callable[[np.ndarray], Union[float, List[float]]]:
        """Returns a function handle to evaluates the energy at given parameters for the ansatz.

        This is the objective function to be passed to the optimizer that is used for evaluation.

        Args:
            operator: The operator whose energy to evaluate.
            return_expectation: If True, return the ``ExpectationBase`` expectation converter used
                in the construction of the expectation value. Useful e.g. to evaluate other
                operators with the same expectation value converter.


        Returns:
            Energy of the hamiltonian of each parameter, and, optionally, the expectation
            converter.

        Raises:
            RuntimeError: If the circuit is not parameterized (i.e. has 0 free parameters).

        """
        num_parameters = self.ansatz.num_parameters
        if num_parameters == 0:
            raise RuntimeError("The ansatz must be parameterized, but has 0 free parameters.")

        expect_op, expectation = self.construct_expectation(
            self._ansatz_params, operator, return_expectation=True
        )
        # tobservable_meas = expectation.convert(StateFn(operator, is_measurement=True))
        # print(operator,tobservable_meas.to_matrix())
        def create_job(operator,params):
            # check if the parameters passed are a range or single value
            if params is not None and len(params.keys()) > 0:
                p_0 = list(params.values())[0]
                if isinstance(p_0, (list, np.ndarray)):
                    num_parameterizations = len(p_0)
                    param_bindings = [
                        {param: value_list[i] for param, value_list in params.items()}  # type: ignore
                        for i in range(num_parameterizations)
                    ]
                    return_as_list = True
                else:
                    num_parameterizations = 1
                    param_bindings = [params]

            else:
                param_bindings = None
                num_parameterizations = 1
            #print('param_bindings',param_bindings)
            results = []
            for circ_params in param_bindings:
                ansatz_in_use = self._ansatz.bind_parameters(circ_params)
                
                transpiled_circ = transpile(ansatz_in_use.decompose(),
                                    basis_gates = self._quantum_instance.backend.configuration().basis_gates,
                                    optimization_level = 0)
                qcirc = qiskit_to_qlm(transpiled_circ)

                job = qcirc.to_job(observable = Observable(operator.num_qubits, matrix=operator.to_matrix()))
                result_temp = self.submit_job(job)
                if isinstance(result_temp,AsyncResult):
                    while result_temp.get_status()!='done':
                        time.sleep(.1)
                    result = result_temp.get_result()
                else:
                    result= result_temp
                results.append(result.value)
            return results

        def energy_evaluation(parameters):
            parameter_sets = np.reshape(parameters, (-1, num_parameters))
            
            #print(expect_op, expectation)
            # Create dict associating each parameter with the lists of parameterization values for it
            param_bindings = dict(zip(self._ansatz_params, parameter_sets.transpose().tolist()))
            #print('param_bindings',param_bindings)
            #start_time = time()
            # sampled_expect_op = self._circuit_sampler.convert(expect_op, params=param_bindings)
            # means = np.real(sampled_expect_op.eval())
            # print('their',means,param_bindings, type(expect_op))
            #MODMOD
            # print('evaluation')
            # print(operator)
            # print(sampled_expect_op.num_qubits)
            # print(param_bindings)
            # print('pre check',operator.num_qubits,operator.to_matrix())
            #print('check',[(type(i),i.oplist) for i in tobservable_meas],tobservable_meas.to_matrix())
            means = np.real(create_job(operator,param_bindings))
            print('mine',means, parameters)
            #MODMOD

            if self._callback is not None:
                parameter_sets = np.reshape(parameters, (-1, num_parameters))
                param_bindings = dict(zip(self._ansatz_params, parameter_sets.transpose().tolist()))
                variance = np.real(expectation.compute_variance(self._circuit_sampler.convert(expect_op, params=param_bindings)))
                estimator_error = np.sqrt(variance / self.quantum_instance.run_config.shots)
                for i, param_set in enumerate(parameter_sets):
                    self._eval_count += 1
                    self._callback(self._eval_count, param_set, means[i], estimator_error[i])
            else:
                self._eval_count += len(means)

            #end_time = time()
           

            return means if len(means) > 1 else means[0]

        if return_expectation:
            return energy_evaluation, expectation

        return energy_evaluation


# vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
#the problem
quantum_instance = QuantumInstance(backend=backend_localQiskit[1], shots=shots,  max_credits=None)
vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
from qiskit_mod.qiskit_nat import VHA
vha_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz=VHA(excitations='sd', trotter_steps=2, only_excitations=True))

calc = GroundStateEigensolver(qubit_converter, vha_solver)

calc.solver._vqe = VQE_MyQLM(ansatz=None,
            quantum_instance=calc.solver._quantum_instance,
            optimizer=calc.solver._optimizer,
            initial_point=calc.solver._initial_point,
            gradient=calc.solver._gradient,
            expectation=calc.solver._expectation,
            include_custom=calc.solver._include_custom)
# calc.solver._vqe.submit_job= 'xxx'
from qiskit_nature.algorithms import AdaptVQE
#test adaptVQE
adapt_calc = AdaptVQE(qubit_converter, vqe_solver, threshold=0.001)
adapt_calc.solver._vqe = VQE_MyQLM(ansatz=None,
            quantum_instance=calc.solver._quantum_instance,
            optimizer=calc.solver._optimizer,
            initial_point=calc.solver._initial_point,
            gradient=calc.solver._gradient,
            expectation=calc.solver._expectation,
            include_custom=calc.solver._include_custom)


In [ ]:
import numpy as np
from qat.plugins import Junction
from qat.qpus import get_default_qpu
from qat.core import Result
from qat.core.plugins import AbstractPlugin





class IterativeExploration(Junction):
    def __init__(self, method, es_problem):
        super(IterativeExploration, self).__init__()
        #print(method)
        self.method = method
        self.problem = es_problem

    def run(self, initial_job, meta_data):

        self.method.solver._vqe.submit_job = self.execute

        result = self.method.solve(self.problem)


        print(result.total_energies)
        return Result(value=result.total_energies[0], meta_data=meta_data)
    def get_specs(x):
        return

# class MyPlugin(AbstractPlugin):
#     def __init__(self, method, es_problem):
#         # super(IterativeExploration, self).__init__()
#         print(method)
#         self.method = method
#         self.problem = es_problem
#     def compile(self, batch, hardware_specs):
#         result = self.method(self.problem)
#         print(result.total_energies)
#         return batch

#     def post_process(self, batch_result):
#         # do something with the execution results
#         for result in batch_result.results:
#             print('Result of size', len(result.raw_data))
#         return batch_result

#     def do_post_processing(self):
#         return False


#modfify the solver



# Building a simple stack
qpu = get_default_qpu()
stack = IterativeExploration(adapt_calc, es_problem) | qpu
# stack = MyPlugin(calc.solve, es_problem) | qpu


# and a simple job
from qat.core import Observable
from qat.lang.AQASM import Program, RY

prog = Program()
qbits = prog.qalloc(1)
prog.apply(RY(prog.new_var(float, r"\beta")), qbits)
job = prog.to_circ().to_job(observable=Observable.sigma_z(0, 1))
#/ and a simple job
start = time.time()
calculation = stack.submit(job)
end = time.time()

print('Result',calculation.value,' ,exec_time=', end - start)

In [ ]:
adapt_calc.solve(es_problem).total_energies

In [ ]:
remote = True

if remote:
    from qat.qlmaas import QLMaaSConnection
    from qlmaas.qpus import LinAlg
    from qlmaas.plugins import Nnizer,ScipyMinimizePlugin
    qpu = LinAlg()
    stack = IterativeExploration(adapt_calc, es_problem) | qpu

    start = time.time()
    result_remote = stack.submit(job)
    end = time.time()
    print('Result',result_remote.value,' ,exec_time=', end - start)

In [ ]:


quantum_instance = QuantumInstance(backend=backend_localQiskit[1], shots=shots,  max_credits=None)
vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, ansatz= ansatz_in_use)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)
remote_qpu = LinAlg()
stack = build_QLM_stack(calc,es_problem,remote_qpu)

In [ ]:
solution = stack.submit(simple_qlm_job())
print(solution.value)

In [ ]:
sol = calc.solve(es_problem)
print(sol.total_energies)

In [ ]:
stack.submit_job(simple_qlm_job())

In [ ]:
junk = IterativeExploration(adapt_calc, es_problem)

In [ ]:
junk.get_specs()

In [ ]:
remote_qpu.conn.port

In [ ]:
remote_qpu.conn.connection.host

In [ ]:
import qlmaas

In [ ]:
qlmaas.plugins.__all__


In [ ]:
from qlmaas.plugins import UploadedPlugin

stack = UploadedPlugin(IterativeExploration(calc,es_problem))

In [ ]:
print(UploadedPlugin.__doc__)

In [ ]:
from qlmaas import __init__


In [ ]:
from qat.bind.qlmaas_QLMHelpers_client import RemoteQLMHelpers

In [ ]:
print(RemoteQLMHelpers.get_qpus.)

In [ ]:
np.max.__str__

In [ ]:
# convert only the ansatz and pass the hamiltonian via matrix to Observables)
from qat.interop.qiskit import qiskit_to_qlm
from qat.core import Observable


from qiskit.compiler import transpile
#first version (obsolete)

def VQEwrapper_to_stack(groundstate_solver, es_problem, stack):
    #if isinstance(groundstate_solver._solver, MinimumEigensolverFactory):
    # this must be called after transformation.transform
    solver = groundstate_solver._solver.get_solver(es_problem, groundstate_solver._qubit_converter)
    #solver = groundstate_solver.solver
    ansatz_in_use = solver.ansatz
    #backend = solver.quantum_instance.backend
    backend = QPUToBackend(stack.qpu)

    transpiled_circ = transpile(ansatz_in_use.decompose(),
                                basis_gates = backend.configuration().basis_gates,
                                optimization_level = 0)
    qcirc = qiskit_to_qlm(transpiled_circ)    

    # Convert the observable/hamiltonian
    
    fermionic_hamiltonian = es_problem.second_q_ops()[0]
    hamiltonian_spin = groundstate_solver._qubit_converter.convert_match(fermionic_hamiltonian)
    num_qubits = hamiltonian_spin.num_qubits
    # print(num_qubits, hamiltonian_spin)
    job = qcirc.to_job(observable = Observable(num_qubits, matrix=hamiltonian_spin.to_matrix()))  

    # Stack to run locally (seems faster)
    
    start = time.time()
    calculation = stack.submit(job)
    end = time.time()

    #print("Final energy :", calculation.value , ' , exec time =', end - start)      
    # Convert to qiskit results format        
    raw_result = solver.compute_minimum_eigenvalue.__annotations__['return']()   
    raw_result.eigenvalue =  calculation.value
    raw_result.optimal_parameters = solver.ansatz.parameters
    dict_param = {}
    for i,par in enumerate(solver.ansatz.parameters):
        dict_param[par] = calculation.parameter_map[str(par)]
    raw_result.eigenstate = solver._get_eigenstate(dict_param)
    
    result = es_problem.interpret(raw_result)
    return result
    
                    